In [1]:
import pandas as pd
import numpy as np
from textblob.tokenizers import WordTokenizer as WT
from scipy.sparse import lil_matrix as lm
from textblob import TextBlob
from tqdm import tqdm
import collections
from sklearn.preprocessing import normalize
import re
from time import time
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_rows', 1000)

In [2]:
df = pd.read_csv("FOX.csv",sep="\t")

/home/caleb/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
class TweetGenerator:
    
    def __init__(self, df):
        
        self.rawtweets = df
        self.regexReplace()
        self.handleQuotes()
        self.splitWords()
        self.buildMat()

    def regexReplace(self):
        
        tweets = self.rawtweets.str.replace("…", "...").str.replace(r"https?.*", "", regex=True).str.replace("&amp;", "&")
        tweets = tweets.str.replace("w/", "with ").str.replace(r".*@.*", "", regex=True).str.replace(r"[^A-Za-z0-9:;,\.&\'\"\-!\(\)\[\]]", " ", regex=True)
        tweets = tweets.str.replace(r"^[^a-z]*:([^a-z]*:)?[^A-Za-z0-9]", "", regex=True).str.replace(r"\s+", " ", regex=True)
        tweets = tweets.str.replace("::", ":").str.replace(r"^(\s|\t)*", "", regex=True).str.replace(r"(\s|\t)*$", "", regex=True)        
        
        self.cleanedTweets = tweets.drop(np.arange(tweets.shape[0])[np.array([len(i) == 0 for i in tweets.values])]).reset_index()["tweet"]
        self.size = self.cleanedTweets.shape[0]
        
    def handleQuotes(self):
        quoteLists = self.cleanedTweets.str.findall(r"\s\'[^']*\'").values
        newSeries = []
        count = 0 
        for i in quoteLists:
            if len(i) < 1:
                newSeries += [self.cleanedTweets[count]]
                count += 1
                continue
            for j in i:
                temp = self.cleanedTweets[count].replace(j, " " + j[1:].replace("\'", "@").replace(" ", "_")) 
            newSeries += [temp]
            count += 1
        self.newSeries = newSeries
        
    def splitWords(self):
        
        self.tokenized = []        
        master_blob = TextBlob("%".join(self.newSeries)).tags
        
        blobs = []
        count = 0
        while count < len(master_blob):
            temp = []
            while (count < len(master_blob)) and (master_blob[count][0] != "%"):
                temp += [master_blob[count]]
                count += 1
            blobs += [temp]
            count += 1

        count = 0
        for text in self.newSeries:
            temp = ["^*^$TART"] + [i[0] + "^*^" + i[1] for i in blobs[count]] + ["^*^$TOP"]
            count += 1
            for i in range(1, len(temp)-2):
                if temp[i][0] == "@":
                    temp = temp[:i] + ["\'" + temp[i+1].replace("_", " ").replace("^*^", "\'^*^")] + temp[i+3:]
                    break
            self.tokenized += [temp]
        
        self.words = set()
        pairs, trios = [], set()
        for sentence in self.tokenized:
            n = len(sentence)
            self.words.add(sentence[0])
            for i in range(n-1):
                pairs += [(sentence[i], sentence[i+1])]
                try:
                    trios.add((sentence[i], sentence[i+2]))
                except:
                    n = len(sentence)
                self.words.add(sentence[i+1])
        counter = collections.Counter(pairs)
        self.trios = trios
        self.most_common = set(counter.most_common())
    
    def buildMat(self):
        
        self.n = len(self.words)
        self.word_map = dict(zip(list(self.words), np.arange(self.n)))
        mat = lm((self.n,self.n))
        
        for i in self.most_common:
            words, freq = i
            left, right = words
            mat[self.word_map[left], self.word_map[right]] = freq
        self.mat = normalize(mat, norm='l1', axis=1)
        
    def makeTweets(self, num_tweets=1, min_words=4):
        
        first_ind = self.word_map["^*^$TART"]
        final_ind = self.word_map["^*^$TOP"]
        self.number_map = inv_map = {v: w for w, v in self.word_map.items()}
        
        all_headlines = []
        
        while 1:
            chance = chances
            failed = False
            headline = []
            first = True
            while 1:
                if first:
                    first = False
                    last_ind = np.random.choice(np.arange(self.n), p=self.mat.getrow(first_ind).toarray()[0])
                else:
                    temp = last_ind
                    last_ind = np.random.choice(np.arange(self.n), p=self.mat.getrow(last_ind).toarray()[0])
                    
                next_word = self.number_map[last_ind]
                
                if last_ind == final_ind:
                    if len(headline) <= min_words:
                        failed = True                      
                    break
                else:
                    headline += [next_word]
            if failed:
                continue
            
            all_headlines += [" ".join([i.split("^*^")[0] for i in headline])]
            if len(all_headlines) == num_tweets:
                break
        return all_headlines
        
gf = TweetGenerator(df.tweet)

In [9]:
gf.makeTweets(5)

["Bush 's Lawyer for Obama s team at Radio Address Liquidity Upgrades Electronics Inc Announces Significant Sanctions Against Infomercial Airs First Quarter",
 "JSE All-Share Index Investors Wait for doctors by angry mob attack at a bit of life with Usama Bin Laden 'May Be Dead' Pakistan injuring 14 Billion",
 'Upskirt photo of the Dogs The Year Ago President Obama Botches Surgery',
 'AARP Endorsement of these real Hollywood',
 'Israeli archaeologist says she was shocked that it scrambled in court to CHP Redding California.']